In [90]:
import numpy as np
import scipy as sp
import pandas as pd
import random
import matplotlib.pyplot as plt
import  networkx as nx
import tensorflow as tf

%matplotlib inline
tf.__version__

'0.10.0'

In [158]:
def balanced_stochastic_blockmodel(communities=2, groupsize=3, p_in=1.0, p_out=0.0):
    #gives dense adjacency matrix representaiton of randomly generated SBM with balanced community size

    G = nx.planted_partition_graph(l=communities, k=groupsize, p_in=p_in, p_out =p_out)
    A = nx.adjacency_matrix(G).todense()
    
    return A


communities = 2 #number of communities, chance to 
group_size = 6 #number of nodes in each communitites (balanced so far)
dim_graph = communities*group_size
A = balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.45, p_out=0.01)
print A

data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.5, p_out=0.2)).astype(float) for i in range(10)]

A = balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.15, p_out=0.01)

Adj = tf.cast(A, dtype = tf.float32)

[[0 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 1 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 0 0 0 0 1 1 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]]


In [92]:
?tf.placeholder

In [169]:
dim_graph = 4
communities = 2

communities = 2 #number of communities, chance to 
group_size = 2 #number of nodes in each communitites (balanced so far)


Adj = tf.placeholder(tf.float64)#, shape=[4,4])

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

r =  tf.Variable(tf.random_normal(shape=[1], mean=0.0,
                                 stddev=1.0, dtype=tf.float64,
                                 seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 


eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

#k = tf.Variable(tf.random_uniform([1], minval=0, maxval=dim_graph, dtype=tf.float32, seed=None, name=None))
#k = tf.floor(k)
#dear god it is impossible to convert a tensor to an integer!!!! hard code for now...

k = communities 


true_assignment_a = tf.concat(0, [tf.zeros([communities], dtype=tf.float32),
                                      tf.ones([communities], dtype=tf.float32)])
true_assignment_b = tf.concat(0, [tf.ones([communities], dtype=tf.float32),
                                      tf.zeros([communities], dtype=tf.float32)])



eigenvec_neg = tf.nn.softmax(tf.slice(eigenvec, [0,0], [dim_graph, 2]))#tf.argmax(tf.nn.softmax(tf.slice(Bethe_Hesse_neg, [0,0], [dim_graph, 2])), 1) #this is where we are supposed to take the eigenspace that corresponds to negative eigenvalues, however let's just learn this...
eigenvec_neg = tf.cast(tf.reduce_sum(eigenvec_neg, 1), dtype = tf.float32)


loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec_neg, true_assignment_a))), 
              tf.reduce_sum(tf.square(tf.sub(eigenvec_neg, true_assignment_b))))
#tf.reduce_sum(tf.square(tf.sub(c, true_assignment_a)))
    
    #tf.reduce_sum(tf.square(eigenvec_neg))
#error = tf.div(loss, dim_graph)


optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(loss)


var_grad = tf.gradients(loss, [r])

init = tf.initialize_variables([r])

with tf.Session() as sess:
    sess.run(init)
    a, d, c =  sess.run([eigenvec_neg, r, var_grad], feed_dict = {Adj: data[1]})
    print a, d, c 
    



InvalidArgumentError: Incompatible shapes: [4,4] vs. [12,12]
	 [[Node: gradients_77/sub_83_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](gradients_77/sub_83_grad/Shape, gradients_77/sub_83_grad/Shape_1)]]
Caused by op u'gradients_77/sub_83_grad/BroadcastGradientArgs', defined at:
  File "/Users/xiangli/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/xiangli/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()


In [94]:

tf.reset_default_graph() #the above works without switching out the eigenvalues, on the nightly binary

In [133]:
communities = 2 #number of communities, chance to 
group_size = 3 #number of nodes in each communitites (balanced so far)
A = balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.95, p_out=0.01)
dim_graph = communities*group_size




Adj = tf.cast(A, dtype = tf.float32)

Diag = tf.diag(tf.reduce_sum(Adj,0))

r =  tf.Variable(tf.random_normal(shape=[1], mean=0.0,
                                 stddev=3.0, dtype=tf.float32,
                                 seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph]))-tf.mul(r, Adj)+Diag 


eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

#k = tf.Variable(tf.random_uniform([1], minval=0, maxval=dim_graph, dtype=tf.float32, seed=None, name=None))
#k = tf.floor(k)
#dear god it is impossible to convert a tensor to an integer!!!! hard code for now...

k = communities 


true_assignment_a = tf.concat(0, [tf.zeros([group_size], dtype=tf.float32),
                                      tf.ones([group_size], dtype=tf.float32)])
true_assignment_b = tf.concat(0, [tf.ones([group_size], dtype=tf.float32),
                                      tf.zeros([group_size], dtype=tf.float32)])



eigenvec_neg = tf.nn.softmax(tf.slice(eigenvec, [0,0], [dim_graph, 2]))#tf.argmax(tf.nn.softmax(tf.slice(Bethe_Hesse_neg, [0,0], [dim_graph, 2])), 1) #this is where we are supposed to take the eigenspace that corresponds to negative eigenvalues, however let's just learn this...
eigenvec_neg = tf.cast(tf.reduce_sum(eigenvec_neg, 1), dtype = tf.float32)


loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec_neg, true_assignment_a))), 
              tf.reduce_sum(tf.square(tf.sub(eigenvec_neg, true_assignment_b))))
#tf.reduce_sum(tf.square(tf.sub(c, true_assignment_a)))
    
    #tf.reduce_sum(tf.square(eigenvec_neg))
#error = tf.div(loss, dim_graph)


optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(loss)


var_grad = tf.gradients(loss, [r])

init = tf.initialize_variables([r])

with tf.Session() as sess:
    sess.run(init)
    print  sess.run([loss, r])
    sess.run(train)
    a, b, c, d =  sess.run([loss, r, eigenvec, var_grad])
    print c, d, b
    



[3.0, array([-6.78981876], dtype=float32)]
[[  5.12835681e-01   0.00000000e+00  -0.00000000e+00  -7.07106411e-01
    4.86826360e-01   0.00000000e+00]
 [ -6.88476264e-01   0.00000000e+00  -0.00000000e+00  -1.71363354e-07
    7.25258827e-01   0.00000000e+00]
 [  5.12835145e-01   0.00000000e+00  -0.00000000e+00   7.07107008e-01
    4.86826181e-01   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00  -8.16496551e-01   0.00000000e+00
    0.00000000e+00   5.77350378e-01]
 [  0.00000000e+00  -7.07106769e-01   4.08248335e-01   0.00000000e+00
    0.00000000e+00   5.77350199e-01]
 [  0.00000000e+00   7.07106769e-01   4.08248365e-01   0.00000000e+00
    0.00000000e+00   5.77350259e-01]] [array([ 0.], dtype=float32)] [-6.78981876]


In [121]:
tf.reset_default_graph()

In [109]:

 #the above works without switching out the eigenvalues, on the nightly binary

In [7]:
#No gradient defined for operation 'SelfAdjointEig_1' (op type: SelfAdjointEig)
tf.__version__ #'0.8.0rc0



#No gradient defined for operation 'SelfAdjointEig_1' (op type: SelfAdjointEig)
tf.__version__ #0.10.0rc0 

#nightly binary
#No gradients provided for any variable: ((None, <tensorflow.python.ops.variables.Variable object at 0x113773150>), (None, <tensorflow.python.ops.variables.Variable object at 0x1143a1990>))
tf.__version__ #'0.10.0rc0'

'0.10.0rc0'

In [6]:
dim_graph = 6
communities = 3

communities = 3 #number of communities, chance to 
group_size = 2 #number of nodes in each communitites (balanced so far)
A = balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.15, p_out=0.01)

Adj = tf.cast(A, dtype = tf.float32)

Diag = tf.diag(tf.reduce_sum(Adj,0))

r =  tf.Variable(tf.random_normal(shape=[1], mean=0.0,
                                 stddev=1.0, dtype=tf.float32,
                                 seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph]))-tf.mul(r, Adj)+Diag 


eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

loss =  tf.reduce_sum(tf.square(eigenvec))

var_grad = tf.gradients(loss, [r])

init = tf.initialize_variables([r])

with tf.Session() as sess:
    sess.run(init)
    a, d, c =  sess.run([eigenvec_neg, r, var_grad])
    print a, d, c 
    



LookupError: No gradient defined for operation 'SelfAdjointEig_1' (op type: SelfAdjointEig)

In [7]:
?tf.self_adjoint_eig

Alright, since eigenspace has no backprop operations, we might as well try the powermethod



In [34]:
dim_graph = 6
communities = 3

communities = 3 #number of communities, chance to 
group_size = 2 #number of nodes in each communitites (balanced so far)
A = balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.15, p_out=0.01)

Adj = tf.cast(A, dtype = tf.float32)

Diag = tf.diag(tf.reduce_sum(Adj,0))

r =  tf.Variable(tf.random_normal(shape=[1], mean=0.0,
                                 stddev=1.0, dtype=tf.float32,
                                 seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph]))-tf.mul(r, Adj)+Diag 


#Power iteration
def euclidean_norm(tensor): #need to have this for tf to work
    square_tensor = tf.square(tensor)
    norm_squared = tf.reduce_sum(square_tensor)
    euclidean_norm = tf.sqrt(norm_squared)
    return euclidean_norm



x = tf.Variable(tf.random_normal(shape = [dim_graph, 1], mean=0.0,
                                 stddev=1.0, dtype=tf.float32,
                                 seed=None, name=None))

eigen_vec_1 = tf.matmul(Bethe_Hesse_neg,x)
eigen_vec_1 = eigen_vec_1/euclidean_norm(eigen_vec_1)

##### however many iternations you want to have

x1 = tf.Variable(tf.random_normal(shape = [dim_graph, 1], mean=0.0,
                                 stddev=1.0, dtype=tf.float32,
                                 seed=None, name=None))


eigen_vec_2 = tf.matmul(Bethe_Hesse_neg, tf.sub(x1, x))
eigen_vec_2 = eigen_vec_2/euclidean_norm(eigen_vec_2)


##### however many iternations you want to have

x2 = tf.Variable(tf.random_normal(shape = [dim_graph, 1], mean=0.0,
                                 stddev=1.0, dtype=tf.float32,
                                 seed=None, name=None))


eigen_vec_3 = tf.matmul(Bethe_Hesse_neg, tf.sub(x2, x1+x))
eigen_vec_3 = eigen_vec_2/euclidean_norm(eigen_vec_3)


eigenspace = tf.gather(e)

#k = tf.Variable(tf.random_uniform([1], minval=0, maxval=dim_graph, dtype=tf.float32, seed=None, name=None))
#k = tf.floor(k)
#dear god it is impossible to convert a tensor to an integer!!!! hard code for now...

k = communities 


true_assignment_a = tf.concat(0, [tf.zeros([3], dtype=tf.float32),
                                      tf.ones([3], dtype=tf.float32)])
true_assignment_b = tf.concat(0, [tf.ones([3], dtype=tf.float32),
                                      tf.zeros([3], dtype=tf.float32)])



eigenvec_neg = tf.nn.softmax(tf.slice(Bethe_Hesse_neg, [0,0], [dim_graph, 2]))#tf.argmax(tf.nn.softmax(tf.slice(Bethe_Hesse_neg, [0,0], [dim_graph, 2])), 1) #this is where we are supposed to take the eigenspace that corresponds to negative eigenvalues, however let's just learn this...
eigenvec_neg = tf.cast(tf.reduce_sum(eigenvec_neg, 1), dtype = tf.float32)


loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec_neg, true_assignment_a))), 
              tf.reduce_sum(tf.square(tf.sub(eigenvec_neg, true_assignment_b))))
#tf.reduce_sum(tf.square(tf.sub(c, true_assignment_a)))
    
    #tf.reduce_sum(tf.square(eigenvec_neg))
#error = tf.div(loss, dim_graph)


optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(loss)


var_grad = tf.gradients(loss, [r])

init = tf.initialize_variables([r, x, x1, x2])

with tf.Session() as sess:
    sess.run(init)
    sess.run(train)
    a, b, c, d, e, f =  sess.run([eigen_vec_2, eigen_vec_3, var_grad, eigen_vec_1, x, Bethe_Hesse_neg])
    print a, b, d
    



[[ 0.77943224]
 [ 0.56888998]
 [ 0.03667445]
 [ 0.25939348]
 [ 0.01242011]
 [ 0.00807854]] [[ 0.32399645]
 [ 0.23647769]
 [ 0.01524493]
 [ 0.10782537]
 [ 0.00516283]
 [ 0.00335811]] [[-0.9248603 ]
 [-0.2698786 ]
 [ 0.13164826]
 [-0.22519536]
 [-0.03377839]
 [ 0.05112474]]


In [37]:
?tf.concat

In [6]:
tf.__version__

'0.10.0rc0'

In [165]:
communities = 6
Adj = tf.placeholder(tf.float64)#, shape=[4,4])

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

r =  tf.Variable(tf.random_normal(shape=[1], mean=0.0,
                                 stddev=2.0, dtype=tf.float64,
                                 seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([communities], dtype=tf.float32),
                                      tf.ones([communities], dtype=tf.float32)])
true_assignment_b = tf.concat(0, [tf.ones([communities], dtype=tf.float32),
                                      tf.zeros([communities], dtype=tf.float32)])



eigenvec = tf.nn.softmax(tf.slice(eigenvec, [0,0], [dim_graph, 2]))#tf.argmax(tf.nn.softmax(tf.slice(Bethe_Hesse_neg, [0,0], [dim_graph, 2])), 1) #this is where we are supposed to take the eigenspace that corresponds to negative eigenvalues, however let's just learn this...
eigenvec = tf.cast(tf.reduce_sum(eigenvec, 1), dtype = tf.float32)


loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_a))), 
              tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_b))))
#tf.reduce_sum(tf.square(tf.sub(c, true_assignment_a)))


optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r])

with tf.Session() as sess:
    for i in range(10):
        sess.run(init)
        sess.run(train)
        print sess.run([r, loss_grad, eigenvec_grad, loss], feed_dict = {Adj: data[i]})
    
    



InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_25' with dtype double
	 [[Node: Placeholder_25 = Placeholder[dtype=DT_DOUBLE, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'Placeholder_25', defined at:
  File "/Users/xiangli/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/xiangli/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.p

In [136]:
?tf.gradients